# 6. Batch Normalization (BN)
本节旨在介绍[Batch Normalization](https://zhuanlan.zhihu.com/p/34879333)及其[实现方法](https://zhuanlan.zhihu.com/p/87117010)，以及[BN对于模型性能的影响](https://mp.weixin.qq.com/s/FFLQBocTZGqnyN79JbSYcQ)。

## 6.1 Batch Normalization原理
随着网络深度的增加，每层特征的分布会逐渐向激活函数输出区间的上下两端靠近（激活函数饱和区间），致使梯度消失。

BN通过批标准化，希望每层特征的分布重新拉回到标准正态分布，1.使得每层网络的输入能尽量保持相同分布以去除Internal Covariate Shifts (ICS); 2.使特征能够处于激活函数的梯度对输入较为敏感的区间，从而使得梯度变大，避免梯度消失的同时加快模型收敛速度。（Relu, resnet等方法都注意到了这个问题）

BN通过保留通道channel的维度，对每个batch中的特征按维度做标准化处理（图片作为样本的话就是按照HW整张图做标准化）：

<p align=center>
<img src="./fig/6-0.png" width=300>
</p>

为方便理解，这里举一个可视化例子：

<p align=center>
<img src="./fig/6-1.png" width=500>
</p>

得到各维标准化后的特征数据

<p align=center>
<img src="./fig/6-2.png" width=500>
</p>

虽然这种归一化操作缓解了ICS问题，但却导致了数据表达能力的缺失，因为改变了原有数据的信息表达，使得原网络学习到的参数信息丢失。因此，BN引入两个可学习的参数$\gamma, \beta$来恢复数据本身的表达能力，对标准化后的数据进行线性变换
$$\tilde{Z}_j=\gamma_j \hat{Z}_j+\beta_j$$
特别地，当$\gamma^2=\sigma^2$且$\beta=\mu$时，该线性变换可以恢复原有数据的分布，实现等价变换。

因此，通过引入可学习的 $\gamma, \beta$以及该线性变换，我们一定程度上保留了输入数据的表达能力，使得网络可以根据loss调整对特征分布的变换，以得到满意的效果。

在测试阶段，BN的可学习参数将被固定，并保留了每个batch的训练数据在网络每一层的$\mu_{batch}$和$\sigma^2_{batch}$，在此基础上无偏估计整体数据分布的均值与协方差用于测试。测试数据将使用$\mu_{test}$和$\sigma^2_{test}$来标准化数据，即：

<p align=center>
<img src="./fig/6-3.png" width=500>
</p>

## 6.2 BN的优势
在此将BN的优势总结为4点，详情可参考[知乎-天雨粟](https://zhuanlan.zhihu.com/p/34879333):
+ BN使得网络中每层输入数据的分布相对稳定，加速模型学习速度
+ BN使得模型对网络中设置的参数（如学习率）不那么敏感，简化调参过程，使得网络学习更加稳定
+ BN允许网络使用饱和性激活函数（例如sigmoid，tanh等），缓解梯度消失问题
+ BN具有一定的正则化效果（Batch对总体的抽样为网络学习增加了随机噪声，类似于drop out关闭神经元给训练带来噪声，提升模型泛化性）

## 6.3 BN的Torch实现与应用

## 6.4 BN对模型性能的影响